# Generate tiled region run file

Automatically creates tiled regions of multiple contiguous FOVs based on user-provided input

In [1]:
import sys
sys.path.append('../')

In [2]:
import json
import os
from skimage.io import imread

from toffy import tiling_utils

# suppress mpl deprecation
import warnings
from matplotlib.cbook import mplDeprecation
warnings.filterwarnings("ignore", category=mplDeprecation)

### Define paths to the JSON data

Define the following parameters to set your home directory:

* `base_dir`: the root folder, `json_tiling_data` should be placed here
* `json_tiling_dir`: the directory containing the information to read and write the FOV info

Define the following input and output paths:

* `region_corners_path`: the JSON file containing the FOVs defining the top-left corner of each region
* `tiled_region_fovs_path`: where to store JSON defining the FOVs for each tiled region
* `moly_path`: the path to the Moly point, needed if you want to insert between FOVs

In [3]:
# define the tiling directory
base_dir = "../creed/data_dir"
json_tiling_dir = os.path.join(base_dir, "json_tiling_data")

# define the prefix of each file
# tma_prefix = "example_tiled_region"
tma_prefix = "example_tma"

# define the slide path
slide_path = os.path.join(json_tiling_dir, "%s_slide.png" % tma_prefix)

In [4]:
# define the input and output files
region_corners_path = os.path.join(json_tiling_dir, 'region_corners.json')
tiled_region_fovs_path = os.path.splitext(region_corners_path)[0] + '_tiled_fovs.json'
moly_path = os.path.join(json_tiling_dir, '%s_moly_point.json' % tma_prefix)
mapping_path = os.path.join(json_tiling_dir, "sample_tiled_region_mapping.json")

### Set tiling parameters

The following parameters need to be defined for each region:

* The FOV defining the top-left corner with its centroid (taken from `region_corners_path`, each entry indicates a different region)
* Number of FOVs along the row and column axes (user prompt)
* row- and column-axis step size (taken from the `fovSizeMicrons` field of each corner FOV in `region_corners_path`, same for row and column)
* Whether to randomize the order of the FOVs or not (user prompt)
* Whether to insert Moly points between regions (user prompt)
* Whether to insert Moly points between FOVs at a specified interval (user prompt). Refer to this link for how Moly point insertion works: https://ark-analysis.readthedocs.io/en/latest/_markdown/ark.mibi.html#ark.mibi.tiling_utils.tiled_region_generate_fov_list

In [5]:
tiling_params = tiling_utils.set_tiled_region_params(region_corners_path)

Using start coordinates of (10000, 45000) in microns for region TheFirst
Using FOV step size of 1000 microns for both row (y) and column (x) axis of region TheFirst


Enter the number of FOVs per row for region TheFirst:  3
Enter the number of FOVs per column for region TheFirst:  4
Randomize FOVs for region TheFirst? Y/N:  N


Using start coordinates of (17500, 30000) in microns for region TheSecond
Using FOV step size of 2000 microns for both row (y) and column (x) axis of region TheSecond


Enter the number of FOVs per row for region TheSecond:  4
Enter the number of FOVs per column for region TheSecond:  3
Randomize FOVs for region TheSecond? Y/N:  N
Insert a moly point between each tiled region? Y/N:  N
Enter the FOV interval size to insert Moly points. If yes, enter the number of FOVs between each Moly point. If no, enter 0:  0


### Create the FOVs for each region

FOV names correspond to their row and column position on the grid of FOVs and are 1-indexed. For example, `R1C1` means the FOV is in row 1 and column 1, `R2C1` would be the next FOV down, and `R1C2` would be the next FOV to the right. The top left FOV is `R1C1`, the bottom right FOV would be `R4C5` for a tiled region of 4x5 dimensions. 

In [6]:
# generate the FOVs in each region
tiled_region_fovs = tiling_utils.generate_tiled_region_fov_list(
    tiling_params,
    moly_path
)

### Interactive FOV deselection

In [7]:
# read the slide image in
slide_data = imread(slide_path)

In [8]:
%matplotlib widget
tiling_utils.tiled_region_interactive_remap(
    tiled_region_fovs,
    tiling_params,
    slide_data,
    mapping_path,
    figsize=(7, 7)
)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to  previous…

About to iterate over all FOVs
Drawing for FOV TheFirst_R1C1
(457, 537)
(472, 537)
(457, 551)
14
15
Drawing for FOV TheFirst_R2C1
(472, 537)
(487, 537)
(472, 551)
14
15
Drawing for FOV TheFirst_R3C1
(487, 537)
(501, 537)
(487, 551)
14
14
Drawing for FOV TheFirst_R4C1
(501, 537)
(516, 537)
(501, 551)
14
15
Drawing for FOV TheFirst_R1C2
(457, 551)
(472, 551)
(457, 566)
15
15
Drawing for FOV TheFirst_R2C2
(472, 551)
(487, 551)
(472, 566)
15
15
Drawing for FOV TheFirst_R3C2
(487, 551)
(501, 551)
(487, 566)
15
14
Drawing for FOV TheFirst_R4C2
(501, 551)
(516, 551)
(501, 566)
15
15
Drawing for FOV TheFirst_R1C3
(457, 566)
(472, 566)
(457, 580)
14
15
Drawing for FOV TheFirst_R2C3
(472, 566)
(487, 566)
(472, 580)
14
15
Drawing for FOV TheFirst_R3C3
(487, 566)
(501, 566)
(487, 580)
14
14
Drawing for FOV TheFirst_R4C3
(501, 566)
(516, 566)
(501, 580)
14
15
Drawing for FOV TheSecond_R1C1
(670, 638)
(700, 638)
(670, 667)
29
30
Drawing for FOV TheSecond_R2C1
(700, 638)
(729, 638)
(700, 667)
29
29
D

In [9]:
# # write the tiled output
# with open(tiled_region_fovs_path, 'w', encoding='utf-8') as trfp:
#     json.dump(tiled_region_fovs, trfp)